In [5]:
!pip install stocktrends

  Preparing metadata (setup.py) ... done
  Created wheel for stocktrends: filename=stocktrends-0.1.5-py3-none-any.whl size=5248 sha256=9c1b2ea01ae099230173fab0ab5ff1f5ab4b08fa0343a620d0f9b876adc4cd55
  Stored in directory: /root/.cache/pip/wheels/95/eb/8e/e9209074104baa9471a9afdea19989fbc76d7276522f3dbca1
Successfully built stocktrends


In [6]:
import yfinance as yf
from stocktrends import Renko

In [7]:
tickers = ["AMZN", "MSFT"]
ohlcv_data = {}
hour_data = {}
renko_data = {}

In [10]:
for ticker in tickers:
  temp = yf.download(ticker, period = '1mo', interval = '5m')
  temp.dropna(how = "any", inplace = True)
  ohlcv_data[ticker] = temp
  temp = yf.download(ticker, period = '1y', interval = '1h')
  temp.dropna(how = "any", inplace = True)
  hour_data[ticker] = temp

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


In [11]:
def ATR(DF, n=14):
    "function to calculate True Range and Average True Range"
    df = DF.copy()
    df["H-L"] = df["High"] - df["Low"]
    df["H-PC"] = abs(df["High"] - df["Adj Close"].shift(1))
    df["L-PC"] = abs(df["Low"] - df["Adj Close"].shift(1))
    df["TR"] = df[["H-L","H-PC","L-PC"]].max(axis=1, skipna=False)
    df["ATR"] = df["TR"].ewm(com=n, min_periods=n).mean()
    return df["ATR"]

In [22]:
def renko_DF(DF, hourly_df):
  df = DF.copy()
  df.drop("Close", axis = 1, inplace = True)
  df.reset_index(inplace = True)
  df.columns = ["date", "open", "high", "low", "close", "volume"]
  df2 = Renko(df)
  df2.brick_size = 3 * round(ATR(hourly_df, 120).iloc[-1],0)
  renko_df = df2.get_ohlc_data()
  return renko_df

In [23]:
for ticker in ohlcv_data:
  renko_data[ticker] = renko_DF(ohlcv_data[ticker], hour_data[ticker])

In [24]:
print(renko_data['AMZN'])

                        date   open   high    low  close  uptrend
0  2024-07-09 09:30:00-04:00  192.0  198.0  192.0  198.0     True
1  2024-07-18 10:30:00-04:00  192.0  192.0  186.0  186.0    False
2  2024-07-25 09:50:00-04:00  186.0  186.0  180.0  180.0    False
3  2024-08-02 09:30:00-04:00  180.0  180.0  174.0  174.0    False
4  2024-08-02 09:30:00-04:00  174.0  174.0  168.0  168.0    False
5  2024-08-02 09:55:00-04:00  168.0  168.0  162.0  162.0    False
6  2024-08-05 09:30:00-04:00  162.0  162.0  156.0  156.0    False
